In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs

In [4]:
# we create 40 separable points
random_state = 2**3
x, y = make_blobs(n_samples=40, centers=2, random_state=random_state)

In [5]:
# fit the model, don't regularize for illustration purposes
model = svm.SVC(kernel='rbf', C=1000)
model.fit(x, y)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [6]:
x_min = x[:, 0].min() * 1.2
x_max = x[:, 0].max() * 1.2
y_min = x[:, 1].min() * 1.2
y_max = x[:, 1].max() * 1.2

# w = model.coef_[0]
# print('coef_', w)
# a = -w[0] / w[1]
# xx = np.linspace(x_min, x_max)
# yy = a * xx - (model.intercept_[0]) / w[1]
# print('intercept_', model.intercept_)
# print(xx)
# print(yy)

# margin = 1 / np.sqrt(np.sum(model.coef_ ** 2))
# yy_down = yy - np.sqrt(1 + a ** 2) * margin
# yy_up = yy + np.sqrt(1 + a ** 2) * margin

In [7]:
from sklearn.metrics.pairwise import rbf_kernel

In [8]:
x_test = np.array([[6.0, 1.0], [8.0, 2.0], [10.0, 10.0]], dtype=np.float32)
pred_test = model.predict(x_test)
print(pred_test)

[1 1 0]


In [9]:
sup_x = model.support_vectors_
dual_alpha = model.dual_coef_
beta = model.intercept_
gamma = model._gamma

print('sup_x', sup_x.shape)
print('dual_alpha', dual_alpha)
print('gamma', gamma)

sup_x (8, 2)
dual_alpha [[-0.87526574 -0.22879476 -0.14720677 -0.51133736  0.69784636  0.1468668
   0.55388271  0.36400875]]
gamma 0.04089030167538266


In [22]:
# rbf = 
for xt in x_test:
    print(rbf_kernel(sup_x, np.reshape(xt, (1, len(xt))), gamma=gamma))

print(rbf_kernel(sup_x, x_test, gamma=gamma))

# check rbf result
# x_repeat = np.repeat([x_test[0]], sup_x.shape[0], axis=0)
# np.array([np.exp(-np.sum((sup_x-x_repeat)**2, axis=1)*gamma)]).T

# nabla0 = 2.0 * gamma * np.sum(((sup_x[:, 0] - x_test[0][0]) * dual_alpha).flatten() * rbf.flatten())
# nabla1 = 2.0 * gamma * np.sum(((sup_x[:, 1] - x_test[0][1]) * dual_alpha).flatten() * rbf.flatten())
# print(nabla0, nabla1)

print('df=', model.decision_function(x_test))

# grad_df = 2.0 * gamma * np.reshape(dual_alpha.flatten() * rbf.flatten(), (1, len(sup_x))) @ (sup_x-x_test[0])
# print(grad_df)



[[0.13478176]
 [0.00839269]
 [0.0614891 ]
 [0.00679962]
 [0.86175509]
 [0.81808781]
 [0.53816572]
 [0.83457418]]
[[0.17985559]
 [0.01442762]
 [0.1498809 ]
 [0.02219986]
 [0.56115562]
 [0.51606503]
 [0.78930284]
 [0.97918965]]
[[0.37298976]
 [0.33866328]
 [0.88704728]
 [0.87995424]
 [0.01606628]
 [0.00269224]
 [0.0243716 ]
 [0.07703762]]
[[0.13478176 0.17985559 0.37298976]
 [0.00839269 0.01442762 0.33866328]
 [0.0614891  0.1498809  0.88704728]
 [0.00679962 0.02219986 0.87995424]
 [0.86175509 0.56115562 0.01606628]
 [0.81808781 0.51606503 0.00269224]
 [0.53816572 0.78930284 0.0243716 ]
 [0.83457418 0.97918965 0.07703762]]
df= [ 1.20393357  1.07982708 -0.91837726]


In [ ]:
y_sign = np.array([[-1 if yy == 0 else 1 for yy in pred_test]], dtype=np.float32).T

delta = grad_df * 2 * y_sign)
adversarial = x_test - delta

In [ ]:
XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
Z = Z.reshape(XX.shape)

figsize = np.array(plt.rcParams["figure.figsize"]) * 2
plt.figure(figsize=figsize.tolist())

plt.contourf(XX, YY, Z, cmap='coolwarm', alpha=0.4, zorder=1)

plt.scatter(x[:, 0], x[:, 1], c=y, cmap='coolwarm', edgecolor='face', s=40, zorder=2)
plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], c=y[model.support_], cmap='coolwarm', edgecolors='white', linewidths=2, s=80, zorder=10)

pred_test = model.predict(x_test)
plt.scatter(x_test[:, 0], x_test[:, 1], c=pred_test, cmap='coolwarm', edgecolor='k', s=80, zorder=20)
plt.scatter(adversarial[:, 0], adversarial[:, 1], c=model.predict(adversarial), cmap='coolwarm', edgecolor='lime', s=80, zorder=20)

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.show()